In [1]:
import pandas as pd
import numpy as np
import json

## Analyze Crash Data

In [2]:
crashes = pd.read_csv('Crashes_in_DC.csv')

C:\Users\Himanshu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
bad_cols= ['LOCATIONERROR', 'LASTUPDATEDATE', 'TODATE', 'MPDGEOX', 'MPDGEOY']
for col in bad_cols:
    crashes.drop(col, inplace=True, axis=1)

In [4]:
crashes.isnull().sum()

X                                 0
Y                                 0
OBJECTID                          0
CRIMEID                           0
CCN                               0
REPORTDATE                     1080
ROUTEID                        3272
MEASURE                        3272
OFFSET                         3272
STREETSEGID                    3272
ROADWAYSEGID                   3272
FROMDATE                          6
MARID                             0
ADDRESS                          57
LATITUDE                         19
LONGITUDE                        19
XCOORD                            0
YCOORD                            0
WARD                              2
EVENTID                           0
MAR_ADDRESS                    8698
MAR_SCORE                         0
MAJORINJURIES_BICYCLIST           0
MINORINJURIES_BICYCLIST           0
UNKNOWNINJURIES_BICYCLIST         0
FATAL_BICYCLIST                   0
MAJORINJURIES_DRIVER              0
MINORINJURIES_DRIVER        

## Analyze INJURIES, FATALITIES and IMPAIRMENTS by LOCATION  
### Location Factors
* Coordinates
* Street Name
* Ward

In [5]:
from tqdm import tqdm_notebook as tqdm

In [6]:
crashes.shape

(204291, 49)

In [7]:
location_cols = ['WARD', 'ROUTEID', 'NEARESTINTSTREETNAME', 'INTAPPROACHDIRECTION']

event_cols = ['MAJORINJURIES_BICYCLIST',
 'MINORINJURIES_BICYCLIST',
 'UNKNOWNINJURIES_BICYCLIST',
 'FATAL_BICYCLIST',
 'MAJORINJURIES_DRIVER',
 'MINORINJURIES_DRIVER',
 'UNKNOWNINJURIES_DRIVER',
 'FATAL_DRIVER',
 'MAJORINJURIES_PEDESTRIAN',
 'MINORINJURIES_PEDESTRIAN',
 'UNKNOWNINJURIES_PEDESTRIAN',
 'FATAL_PEDESTRIAN',]

issues = {}
for col in location_cols:
    issues.setdefault(col, {})
    print(f"Number of unique {col}: {len(crashes[col].unique())}")
    for location in tqdm(crashes[col].unique(), total=len(crashes[col].unique())):
        loc_df = crashes[crashes[col]==location]
        issues[col].setdefault(location, {})
        issues[col][location]['TOTAL_EVENTS'] = loc_df.shape[0]
        for event in event_cols:
            issues[col][location].setdefault(event, loc_df[event].sum())

Number of unique WARD: 10



Number of unique ROUTEID: 8194



Number of unique NEARESTINTSTREETNAME: 2124



Number of unique INTAPPROACHDIRECTION: 9


In [8]:
def create_stats_df(issues_dict, location, rate):
    df = pd.DataFrame(issues_dict[location]).T
    if rate==True:
        for col in df:
            df[f'{col}_RATE'] = (df[col] / df['TOTAL_EVENTS'])*100
            
    return df

In [9]:
direction_df = create_stats_df(issues, 'INTAPPROACHDIRECTION', rate=True)
direction_df

,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,TOTAL_EVENTS,...,MAJORINJURIES_BICYCLIST_RATE,MAJORINJURIES_DRIVER_RATE,MAJORINJURIES_PEDESTRIAN_RATE,MINORINJURIES_BICYCLIST_RATE,MINORINJURIES_DRIVER_RATE,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE
NaN,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
North,3,48,7,74,5945,287,400,10336,884,57237,...,0.129287,10.386638,0.501424,0.698849,18.058249,1.544456,100.0,0.00000,0.232367,0.003494
South,0,25,3,56,2438,82,369,4896,576,31141,...,0.179827,7.828907,0.263318,1.184933,15.722038,1.849652,100.0,0.00000,0.234418,0.000000
East,0,18,2,31,1884,48,187,3314,283,21784,...,0.142306,8.648549,0.220345,0.858428,15.213000,1.299119,100.0,0.00000,0.339699,0.009181
Northwest,0,14,1,7,844,13,64,1684,114,9380,...,0.074627,8.997868,0.138593,0.682303,17.953092,1.215352,100.0,0.00000,0.330490,0.010661
Southeast,0,5,1,10,720,22,61,1604,121,8876,...,0.112663,8.111762,0.247859,0.687247,18.071203,1.363227,100.0,0.00000,0.349256,0.011266
Southwest,0,15,1,8,771,17,72,1987,132,10161,...,0.078732,7.587836,0.167306,0.708592,19.555162,1.299085,100.0,0.00000,0.364137,0.009842
Northeast,1,11,0,8,903,28,56,2090,114,10423,...,0.076753,8.663533,0.268637,0.537273,20.051809,1.093735,100.0,0.00000,0.393361,0.000000
West,3,11,2,41,2600,50,336,5282,520,34637,...,0.118371,7.506424,0.144354,0.970061,15.249589,1.501285,100.0,0.02021,1.804429,0.046193


In [10]:
wards_df = create_stats_df(issues, 'WARD', rate=True)
wards_df.fillna("", inplace=True)
wards_df

,FATAL_BICYCLIST,FATAL_DRIVER,FATAL_PEDESTRIAN,MAJORINJURIES_BICYCLIST,MAJORINJURIES_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_BICYCLIST,MINORINJURIES_DRIVER,MINORINJURIES_PEDESTRIAN,TOTAL_EVENTS,...,MAJORINJURIES_BICYCLIST_RATE,MAJORINJURIES_DRIVER_RATE,MAJORINJURIES_PEDESTRIAN_RATE,MINORINJURIES_BICYCLIST_RATE,MINORINJURIES_DRIVER_RATE,MINORINJURIES_PEDESTRIAN_RATE,TOTAL_EVENTS_RATE,UNKNOWNINJURIES_BICYCLIST_RATE,UNKNOWNINJURIES_DRIVER_RATE,UNKNOWNINJURIES_PEDESTRIAN_RATE
Ward 6,1,23,1,35,2169,68,266,4987,454,29962,...,0.116815,7.23917,0.226954,0.887791,16.6444,1.51525,100,0.00667512,0.814365,0.0433883
Ward 2,2,27,6,100,3191,147,624,5623,825,46730,...,0.213995,6.82859,0.314573,1.33533,12.033,1.76546,100,0.0149797,0.727584,0.0256794
Ward 7,2,35,4,8,2426,83,55,5204,296,24961,...,0.03205,9.71916,0.332519,0.220344,20.8485,1.18585,100,0.0080125,1.32607,0.016025
Ward 3,0,4,0,8,1080,39,75,2333,187,13190,...,0.060652,8.18802,0.295679,0.568613,17.6876,1.41774,100,0,0.515542,0.0075815
Ward 5,1,30,3,32,2831,62,205,6200,385,30682,...,0.104296,9.22691,0.202073,0.668144,20.2073,1.25481,100,0,0.837625,0.0260739
Ward 1,3,13,2,49,1669,77,322,2401,346,18315,...,0.26754,9.11275,0.42042,1.75812,13.1095,1.88916,100,0.02184,0.698881,0.01638
Ward 4,0,12,3,24,1726,56,143,3643,260,17815,...,0.134718,9.68846,0.314342,0.802694,20.4491,1.45944,100,0,0.493966,0.00561325
Ward 8,1,20,1,8,1797,77,55,4144,347,21248,...,0.0376506,8.45727,0.362387,0.258848,19.503,1.63309,100,0.00470633,1.77428,0.0235316
NaN,0,0,0,0,0,0,0,0,0,0,...,,,,,,,,,,
Null,0,0,0,0,192,16,1,271,25,1386,...,0,13.8528,1.1544,0.0721501,19.5527,1.80375,100,0,0,0


In [11]:
wards_dict = wards_df.to_dict(orient='index')

In [12]:
with open('ward_crash_data.json', 'w') as outfile:
    json.dump(wards_dict, outfile)

NameError: name 'json' is not defined

In [ ]:
nearest_street_df = create_stats_df(issues, 'NEARESTINTSTREETNAME', rate=True)

In [ ]:
list(nearest_street_df)

In [ ]:
nearest_street_df.sort_values(by='MAJORINJURIES_BICYCLIST_RATE', ascending=False)